<a href="https://colab.research.google.com/github/fatmaaerr/AMD_Medal_Analiz_Project_MBA/blob/main/AMD_Project_2_Medal_Analiz_and_Model_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>

# **Market Basket Analysis**


---
<br>

## Algorithm for Massive Datasets Project

### University of Milan
### Data Science and Economics


## **Author**: Fatma ER

## Student's ID: 967585

<br>
<br>

<hr>

## Project 2: Market-Basket Analysis

The task is to implement a system finding frequent itemsets (aka market-basket analysis), analyzing one of the two datasets described below.
Medal. <br> «<a href="https://www.kaggle.com/datasets/xhlulu/medal-emnlpt" >MeDAL Dataset dataset is published on Kaggle</a>.

The detector must consider as baskets the strings contained in the text column of the  full-data.csv file in the dataset, using words as items.

<br>
<br>

<hr>

<br>
<br>

# **Dataset download using Kaggle API**

<br>

In [ ]:
! pip install kaggle spark-nlp==3.4.2 pyspark==3.2.0 ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805913 sha256=5f879f603a167de157914c4a2b2692f3e1ca5fc736758fddb3dfc3bb31f81eff
  Stored in directory: /root/.cache/pip/wheels/23/f6/d3/110e53bd43baeb8d7d38049733d48e39cbecd056f01dba7ee8
Successfully built pyspark
time: 996 µs (started: 2023-03-01 15:41:24 +00:00)


In [ ]:
from pyspark.sql import SparkSession
import os
from glob import glob
from pyspark.sql.types import FloatType
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import * 
import string
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas as pd
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from itertools import combinations
import math
import numpy as np
import logging
from joblib import Parallel, delayed, cpu_count
from collections import namedtuple
from itertools import product
import time
from itertools import chain
spark = sparknlp.start(spark32 = True)
sc = spark.sparkContext

time: 1min 2s (started: 2023-03-01 15:41:24 +00:00)


In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
import os

time: 417 ms (started: 2023-03-01 15:42:26 +00:00)


In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.2
!wget -q http://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u352-ga-1~20.04_amd64.deb  404  Not Found [IP: 185.125.190.39 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u352-ga-1~20.04_amd64.deb  404  Not Found [IP: 185.125.190.39 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
tar: spark-3.0.3-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
time: 12.4 s (started: 2023-03-01 15:42:27 +00:00)


time: 12.4 s (started: 2023-03-01 15:42:27 +00:00)


In [ ]:
#!pip install kaggle

time: 709 µs (started: 2023-03-01 15:42:39 +00:00)


In [ ]:
# Kaggle API to download the dataset
import os
kaggle_api = {"username":"fatmaer","key":"195fdbf5ac9e388a70b2497fbd2c2a4c"}
os.environ['KAGGLE_USERNAME'] = kaggle_api['username']
os.environ['KAGGLE_KEY'] = kaggle_api['key']

time: 1.74 ms (started: 2023-03-01 15:42:39 +00:00)


In [ ]:
# Kaggle authentication and download
import kaggle
kaggle.api.authenticate()
!  kaggle datasets download xhlulu/medal-emnlp

100% 6.82G/6.82G [04:22<00:00, 26.4MB/s]
100% 6.82G/6.82G [04:22<00:00, 27.9MB/s]
time: 4min 23s (started: 2023-03-01 15:42:39 +00:00)


In [ ]:
!unzip medal-emnlp

Archive:  medal-emnlp.zip
  inflating: full_data.csv           
  inflating: pretrain_subset/test.csv  
  inflating: pretrain_subset/train.csv  
  inflating: pretrain_subset/valid.csv  
time: 4min 31s (started: 2023-03-01 15:47:03 +00:00)


**Spark Initialisation**

In [ ]:
##!pip install pyspark
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!pip install -q findspark

time: 1.19 ms (started: 2023-03-01 15:51:34 +00:00)


In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

time: 1.24 s (started: 2023-03-01 15:51:34 +00:00)


In [ ]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

time: 15.9 ms (started: 2023-03-01 15:51:35 +00:00)


In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

time: 5.29 ms (started: 2023-03-01 15:51:35 +00:00)


In [ ]:
sc = spark.sparkContext

time: 895 µs (started: 2023-03-01 15:51:35 +00:00)


In [ ]:
full_data = spark.read.csv("full_data.csv", header=True, inferSchema=True)

time: 1min 38s (started: 2023-03-01 15:51:35 +00:00)


In [ ]:
full_data.show(5)

+--------------------+--------------------+--------------------+
|                TEXT|            LOCATION|               LABEL|
+--------------------+--------------------+--------------------+
|alphabisabolol ha...|                  56|           substrate|
|a report is given...|24|49|68|113|137|172|carcinosarcoma|re...|
|the virostatic co...|                  55|           substrate|
|rmi rmi and rmi a...|   25|82|127|182|222|compounds|compoun...|
|a doubleblind stu...|22|26|28|77|90|14...|oxazepam|placebo|...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

time: 294 ms (started: 2023-03-01 15:53:14 +00:00)


In [ ]:
full_data.printSchema()


root
 |-- TEXT: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- LABEL: string (nullable = true)

time: 19.3 ms (started: 2023-03-01 15:53:14 +00:00)


In [ ]:
# Get row count
rows = full_data.count()

# Get columns count
cols = len(full_data.columns)

print(f"DataFrame Dimensions : {(rows,cols)}")
print(f"DataFrame Rows count : {rows}")
print(f"DataFrame Columns count : {cols}")

DataFrame Dimensions : (14393619, 3)
DataFrame Rows count : 14393619
DataFrame Columns count : 3
time: 1min 6s (started: 2023-03-01 15:53:14 +00:00)


In [ ]:
from pyspark.sql.functions import desc

time: 1.01 ms (started: 2023-03-01 15:54:21 +00:00)


In [ ]:
counts = full_data.groupBy("LABEL").count().orderBy(desc("count"))

time: 70.2 ms (started: 2023-03-01 15:54:21 +00:00)


In [ ]:
# Count on Groupped Data
full_data.groupBy("LABEL").count().orderBy(desc("count")).show(10)

+-----------+------+
|      LABEL| count|
+-----------+------+
|      study|294977|
|      after|114472|
|    factors| 71336|
|development| 66812|
|     cancer| 61340|
|      model| 58257|
|     levels| 50882|
|   function| 50024|
|   specific| 44586|
|   approach| 43411|
+-----------+------+
only showing top 10 rows

time: 1min 49s (started: 2023-03-01 15:54:21 +00:00)


In [ ]:
# Count on Groupped Data
full_data.groupBy("TEXT").count().orderBy(desc("count")).show(10)

+--------------------+-----+
|                TEXT|count|
+--------------------+-----+
|    retrospective T0|   82|
|  a retrospective T0|   77|
|retrospective coh...|   56|
|                  SR|   56|
|   crosssectional T0|   50|
|prospective cohor...|   44|
|      prospective T0|   21|
|       randomized CT|   17|
|    mo disulfide mos|   15|
|retrospective cli...|   15|
+--------------------+-----+
only showing top 10 rows

time: 5min 5s (started: 2023-03-01 15:56:10 +00:00)


In [ ]:

Study_data = full_data.filter( (full_data.LABEL == "study"))

time: 90.5 ms (started: 2023-03-01 16:01:16 +00:00)


In [ ]:
Study_data.show(10)

+--------------------+--------+-----+
|                TEXT|LOCATION|LABEL|
+--------------------+--------+-----+
|the behaviour of ...|      52|study|
|fast reaction tec...|       6|study|
|the role of the d...|      75|study|
|this study uses a...|      37|study|
|a statistical T0 ...|       2|study|
|the increasing us...|      78|study|
|observation of th...|      33|study|
|this T0 investiga...|       1|study|
|in a T0 of alcoho...|       2|study|
|the authors repor...|      31|study|
+--------------------+--------+-----+
only showing top 10 rows

time: 236 ms (started: 2023-03-01 16:01:16 +00:00)


In [ ]:
# Get row count
Study_data.count()

294977

time: 1min 23s (started: 2023-03-01 16:01:16 +00:00)


In [ ]:
# Count on Groupped Data
Study_data.groupBy("TEXT").count().orderBy(desc("count")).show(10)

+--------------------+-----+
|                TEXT|count|
+--------------------+-----+
|    retrospective T0|   82|
|  a retrospective T0|   77|
|retrospective coh...|   56|
|   crosssectional T0|   50|
|prospective cohor...|   44|
|      prospective T0|   21|
|a retrospective c...|   15|
|retrospective cli...|   15|
|    a prospective T0|   14|
|      casecontrol T0|   13|
+--------------------+-----+
only showing top 10 rows

time: 1min 29s (started: 2023-03-01 16:02:40 +00:00)


In [ ]:
from pyspark.sql import Row

Study_data.dropDuplicates(['TEXT']).show(20)


+--------------------+--------+-----+
|                TEXT|LOCATION|LABEL|
+--------------------+--------+-----+
|T0 a novel method...|       0|study|
|T0 aimed to asses...|       0|study|
|T0 aiming to cros...|       0|study|
|T0 and characteri...|       0|study|
|T0 and comparison...|       0|study|
|T0 and profound a...|       0|study|
|T0 antigenbinding...|       0|study|
|T0 by sem of the ...|       0|study|
|T0 conducted in r...|       0|study|
|T0 design literat...|       0|study|
|T0 designs involv...|       0|study|
|T0 etiologic stru...|       0|study|
|T0 examined perce...|       0|study|
|T0 feasibility of...|       0|study|
|T0 for influence ...|       0|study|
|T0 functional cha...|       0|study|
|T0 goals were fol...|       0|study|
|T0 goals were to ...|       0|study|
|T0 group consiste...|       0|study|
|T0 gs was a rando...|       0|study|
+--------------------+--------+-----+
only showing top 20 rows

time: 1min 37s (started: 2023-03-01 16:04:09 +00:00)


In [ ]:
Study_data.count()

294977

time: 1min 21s (started: 2023-03-01 16:05:46 +00:00)


In [ ]:

# Count on Groupped Data
Study_data.groupBy("TEXT","LOCATION").count().orderBy(desc("count")).show(10)

+--------------------+--------+-----+
|                TEXT|LOCATION|count|
+--------------------+--------+-----+
|    retrospective T0|       1|   82|
|  a retrospective T0|       2|   77|
|retrospective coh...|       2|   56|
|   crosssectional T0|       1|   50|
|prospective cohor...|       2|   44|
|      prospective T0|       1|   21|
|a retrospective c...|       3|   15|
|retrospective cli...|       2|   15|
|    a prospective T0|       2|   14|
| a crosssectional T0|       2|   13|
+--------------------+--------+-----+
only showing top 10 rows

time: 1min 31s (started: 2023-03-01 16:07:08 +00:00)


In [ ]:
Study_data=Study_data.dropDuplicates(["TEXT","LOCATION"])
Study_data.show(10)

+--------------------+--------+-----+
|                TEXT|LOCATION|LABEL|
+--------------------+--------+-----+
|                  T0|       0|study|
|T0 a novel method...|       0|study|
|T0 aiming to cros...|       0|study|
|T0 aims include d...|       0|study|
|T0 aims to evalua...|       0|study|
|T0 aims to evalua...|       0|study|
|T0 and characteri...|       0|study|
|T0 and comparison...|       0|study|
|T0 by sem of the ...|       0|study|
|T0 concerned prin...|       0|study|
+--------------------+--------+-----+
only showing top 10 rows

time: 1min 39s (started: 2023-03-01 16:08:40 +00:00)


In [ ]:
Study_data.count()

294266

time: 1min 29s (started: 2023-03-01 16:10:19 +00:00)


294977 duplicate data delete 294266 

In [ ]:
Study_data.show(5)

+--------------------+--------+-----+
|                TEXT|LOCATION|LABEL|
+--------------------+--------+-----+
|                  T0|       0|study|
|T0 a novel method...|       0|study|
|T0 aiming to cros...|       0|study|
|T0 aims include d...|       0|study|
|T0 aims to evalua...|       0|study|
+--------------------+--------+-----+
only showing top 5 rows

time: 1min 30s (started: 2023-03-01 16:11:49 +00:00)


<hr>

<br>
<br>

# Data Cleaning and Text Processing

<br>


In [ ]:
from pyspark.sql.functions import col, lit, when
from pyspark.sql.types import StructType, StructField, StringType,IntegerType
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import collect_set
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col, udf
from pyspark.sql.types import ArrayType, StringType
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

time: 1.05 s (started: 2023-03-01 16:13:19 +00:00)


In [ ]:
# Start Spark session
spark = SparkSession.builder.appName("TextMining").getOrCreate()

time: 11.3 ms (started: 2023-03-01 16:13:20 +00:00)


In [ ]:
# Convert text to lowercase and create new column for processed text
Study_data = Study_data.withColumn("processed_text", lower(col("text")))

time: 26.9 ms (started: 2023-03-01 16:13:20 +00:00)


In [ ]:
Study_data.show(5)

+--------------------+--------+-----+--------------------+
|                TEXT|LOCATION|LABEL|      processed_text|
+--------------------+--------+-----+--------------------+
|                  T0|       0|study|                  t0|
|T0 a novel method...|       0|study|t0 a novel method...|
|T0 aiming to cros...|       0|study|t0 aiming to cros...|
|T0 aims include d...|       0|study|t0 aims include d...|
|T0 aims to evalua...|       0|study|t0 aims to evalua...|
+--------------------+--------+-----+--------------------+
only showing top 5 rows

time: 1min 29s (started: 2023-03-01 16:13:20 +00:00)


In [ ]:
Study_data.cache()

DataFrame[TEXT: string, LOCATION: string, LABEL: string, processed_text: string]

time: 78.5 ms (started: 2023-03-01 16:14:49 +00:00)


In [ ]:
from pyspark.sql.functions import regexp_replace, trim, col, lower

def removePunctuation(column):
   return lower(trim(regexp_replace(column,'\\p{Punct}',''))).alias('sentence')

Study_data = Study_data.select("LOCATION",removePunctuation(col('processed_text')).alias("processed_text"))
Study_data.show(5)

+--------+--------------------+
|LOCATION|      processed_text|
+--------+--------------------+
|       0|t0 of molecular h...|
|       0|t0 of the impacts...|
|       1|a t0 has been und...|
|       1|a t0 in this issu...|
|       1|a t0 of visiting ...|
+--------+--------------------+
only showing top 5 rows

time: 1min 31s (started: 2023-03-01 16:14:50 +00:00)


In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import trim,split,explode,col


tokenizer = Tokenizer(inputCol='processed_text', outputCol='words_token')
Study_data = Study_data.withColumn("processed_text", trim(col("processed_text")))
Study_data = tokenizer.transform(Study_data)
Study_data.show(20)

+--------+--------------------+--------------------+
|LOCATION|      processed_text|         words_token|
+--------+--------------------+--------------------+
|       0|t0 of molecular h...|[t0, of, molecula...|
|       0|t0 of the impacts...|[t0, of, the, imp...|
|       1|a t0 has been und...|[a, t0, has, been...|
|       1|a t0 in this issu...|[a, t0, in, this,...|
|       1|a t0 of visiting ...|[a, t0, of, visit...|
|       1|a t0 of women wit...|[a, t0, of, women...|
|       1|a t0 was carried ...|[a, t0, was, carr...|
|      68|a best evidence t...|[a, best, evidenc...|
|       3|a clinical interv...|[a, clinical, int...|
|     222|a commonly accept...|[a, commonly, acc...|
|       2|a comparative t0 ...|[a, comparative, ...|
|       2|a controlled t0 o...|[a, controlled, t...|
|       2|a crosssectional ...|[a, crosssectiona...|
|     115|a description is ...|[a, description, ...|
|      57|a detailed analys...|[a, detailed, ana...|
|      86|a factorial desig...|[a, factorial, 

In [ ]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
Study_data = remover.transform(Study_data).select("LOCATION","processed_text",'words_clean')
Study_data.select("words_clean").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import expr

Study_data = Study_data.withColumn("words_clean", expr("filter(words_clean, elem -> elem != '')"))
Study_data.show(10)

+--------+--------------------+--------------------+
|LOCATION|      processed_text|         words_clean|
+--------+--------------------+--------------------+
|       0|t0 of molecular h...|[t0, molecular, h...|
|       0|t0 of the impacts...|[t0, impacts, bio...|
|       1|a t0 has been und...|[t0, undertaken, ...|
|       1|a t0 in this issu...|[t0, issue, molec...|
|       1|a t0 of visiting ...|[t0, visiting, pa...|
|       1|a t0 of women wit...|[t0, women, low, ...|
|       1|a t0 was carried ...|[t0, carried, inf...|
|      68|a best evidence t...|[best, evidence, ...|
|       3|a clinical interv...|[clinical, interv...|
|     222|a commonly accept...|[commonly, accept...|
+--------+--------------------+--------------------+
only showing top 10 rows

time: 313 ms (started: 2023-03-01 16:16:22 +00:00)


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType,StringType

get_uniques=udf(lambda x: list(set(x)), ArrayType(StringType()))

Study_data = Study_data.withColumn("words_clean", get_uniques(Study_data.words_clean))
Study_data.show(10)

+--------+--------------------+--------------------+
|LOCATION|      processed_text|         words_clean|
+--------+--------------------+--------------------+
|       0|t0 of molecular h...|[heterogeneity, k...|
|       0|t0 of the impacts...|[disputed, expose...|
|       1|a t0 has been und...|[obtained, animal...|
|       1|a t0 in this issu...|[protein, inheren...|
|       1|a t0 of visiting ...|[commonly, fewer,...|
|       1|a t0 of women wit...|[deaths, reliable...|
|       1|a t0 was carried ...|[concentration, d...|
|      68|a best evidence t...|[using, papers, f...|
|       3|a clinical interv...|[professionals, r...|
|     222|a commonly accept...|[technique, bacte...|
+--------+--------------------+--------------------+
only showing top 10 rows

time: 1.63 s (started: 2023-03-01 16:16:22 +00:00)


In [ ]:
Study_data = Study_data.drop("LOCATION")
Study_data.show(5)

+--------------------+--------------------+
|      processed_text|         words_clean|
+--------------------+--------------------+
|t0 of molecular h...|[heterogeneity, k...|
|t0 of the impacts...|[disputed, expose...|
|a t0 has been und...|[obtained, animal...|
|a t0 in this issu...|[protein, inheren...|
|a t0 of visiting ...|[commonly, fewer,...|
+--------------------+--------------------+
only showing top 5 rows

time: 302 ms (started: 2023-03-01 16:16:24 +00:00)


# Market Basket Analysis

Basket Build

In [ ]:
from pyspark.sql.functions import collect_set

time: 1.2 ms (started: 2023-03-01 16:16:24 +00:00)


In [ ]:
slen = udf(lambda s: len(s), IntegerType())
Study_data = Study_data.withColumn("word_count", slen(Study_data.words_clean))

time: 57.9 ms (started: 2023-03-01 16:16:24 +00:00)


In [ ]:
Study_data.cache()

DataFrame[processed_text: string, words_clean: array<string>, word_count: int]

time: 72.8 ms (started: 2023-03-01 16:16:25 +00:00)


In [ ]:
Study_data.show(10)

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|t0 of molecular h...|[heterogeneity, k...|        73|
|t0 of the impacts...|[disputed, expose...|        53|
|a t0 has been und...|[obtained, animal...|        25|
|a t0 in this issu...|[protein, inheren...|        24|
|a t0 of visiting ...|[commonly, fewer,...|        58|
|a t0 of women wit...|[deaths, reliable...|        43|
|a t0 was carried ...|[concentration, d...|        72|
|a best evidence t...|[using, papers, f...|        55|
|a clinical interv...|[professionals, r...|        58|
|a commonly accept...|[technique, bacte...|        82|
+--------------------+--------------------+----------+
only showing top 10 rows

time: 858 ms (started: 2023-03-01 16:16:25 +00:00)


In [ ]:
# Count on Groupped Data
Study_data.orderBy(desc("word_count")).show(10)

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|the areolate orie...|[thailand, watten...|       325|
|pseudoxanthoma el...|[eight, household...|       236|
|early biotechnolo...|[protein, develop...|       222|
|conditioned respo...|[arising, name, a...|       221|
|a common goal of ...|[protein, ionexch...|       212|
|the argasid tick ...|[protein, develop...|       210|
|antipatharians co...|[development, app...|       209|
|we shall open our...|[still, developme...|       208|
|electronic patien...|[level, retrieval...|       207|
|the criminal just...|[regime, protein,...|       205|
+--------------------+--------------------+----------+
only showing top 10 rows

time: 59.7 s (started: 2023-03-01 16:16:26 +00:00)


In [ ]:
from pyspark.sql.functions import asc

Study_data.orderBy(asc("word_count")).show(30)

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|                  t0|                [t0]|         1|
|             this t0|                [t0]|         1|
|          in this t0|                [t0]|         1|
|  an experimental t0|  [t0, experimental]|         2|
|      in this t0 two|           [two, t0]|         2|
|   interventional t0|[t0, interventional]|         2|
|this t0 investigated|  [t0, investigated]|         2|
|      descriptive t0|   [t0, descriptive]|         2|
|       an autopsy t0|       [autopsy, t0]|         2|
|  a retrospective t0| [retrospective, t0]|         2|
|        agreement t0|     [t0, agreement]|         2|
|       caseseries t0|    [t0, caseseries]|         2|
|this t0 describre...|[t0, describreatm...|         2|
|      clinimetric t0|   [clinimetric, t0]|         2|
|      reliability t0|   [t0, reliability]|         2|
|       in

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.2
!wget -q http://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u352-ga-1~20.04_amd64.deb  404  Not Found [IP: 91.189.91.39 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u352-ga-1~20.04_amd64.deb  404  Not Found [IP: 91.189.91.39 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
tar: spark-3.0.3-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
time: 9.26 s (started: 2023-03-01 16:17:27 +00:00)


In [ ]:
# Filtering data based on word count (word count should be greated than 5 and less than 50% of distribution (to avoid outlier and extremly lenghthy Text))

Study_data = Study_data.filter((Study_data.word_count > 5) & (Study_data.word_count < Study_data.approxQuantile("word_count", [0.50], 0.25)[0]))
Study_data.show(5)


+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|a phase iii dosee...|[pain, phase, per...|        18|
|a prospective t0 ...|[suspected, quali...|        17|
|a prospective cas...|[leprosy, calcula...|        17|
|acute pancreatiti...|[oedema, fluid, d...|        19|
|agroecological sy...|[systems, prevale...|        16|
+--------------------+--------------------+----------+
only showing top 5 rows

time: 4.95 s (started: 2023-03-01 16:17:37 +00:00)


In [ ]:

# Get row count
rows = Study_data.count()

# Get columns count
cols = len(Study_data.columns)

print(f"DataFrame Dimensions : {(rows,cols)}")
print(f"DataFrame Rows count : {rows}")
print(f"DataFrame Columns count : {cols}")

DataFrame Dimensions : (79006, 3)
DataFrame Rows count : 79006
DataFrame Columns count : 3
time: 2.75 s (started: 2023-03-01 16:17:42 +00:00)


In [ ]:
transactionList = Study_data.select('words_clean').rdd.flatMap(lambda x: x)

time: 168 ms (started: 2023-03-01 16:17:44 +00:00)


In [ ]:
transactionList

PythonRDD[216] at RDD at PythonRDD.scala:53

time: 102 ms (started: 2023-03-01 16:17:45 +00:00)


In [ ]:
from pyspark.sql.functions import asc

Study_data.orderBy(asc("word_count")).show(5)

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|the present t0 ai...|[iran, lymphomas,...|         6|
|this t0 examined ...|[turkey, antibiot...|         6|
|the present t0 wa...|[evaluate, t0, an...|         6|
|to t0 nociceptive...|[headache, t0, no...|         6|
|to t0 the chemica...|[naudea, t0, chem...|         6|
+--------------------+--------------------+----------+
only showing top 5 rows

time: 1.91 s (started: 2023-03-01 15:27:46 +00:00)


In [ ]:
# Count on Groupped Data
Study_data.orderBy(desc("word_count")).show(10)

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|management of lag...|[gel, temporary, ...|        21|
|the presence of t...|[ct, presence, in...|        21|
|many clinical stu...|[account, determi...|        21|
|dexmedetomidine i...|[adjuvant, remain...|        21|
|prisoners have he...|[needs, southern,...|        21|
|psychological fac...|[control, dimensi...|        21|
|the objectives of...|[flow, human, dis...|        21|
|tacrolimus is met...|[new, p, cypa, sw...|        21|
|friedreich ataxia...|[power, generate,...|        21|
|the current t0 ex...|[ed, frequency, e...|        21|
+--------------------+--------------------+----------+
only showing top 10 rows

time: 1.73 s (started: 2023-03-01 15:27:50 +00:00)


## FPGrowth Algoritm

In [ ]:
import time
from tqdm import tqdm
from pyspark.ml.fpm import FPGrowth

time: 1.19 ms (started: 2023-03-01 15:28:02 +00:00)


In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import time

time: 893 µs (started: 2023-03-01 15:28:02 +00:00)


In [ ]:
from pyspark.ml.fpm import FPGrowth

time: 960 µs (started: 2023-03-01 15:28:03 +00:00)


time: 2.66 s (started: 2023-03-01 15:28:03 +00:00)


In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets using support, lift, and confidence.
fpGrowth = FPGrowth(itemsCol="words_clean", minSupport=0.02, minConfidence=0.02)
model = fpGrowth.fit(Study_data)
# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the consequents as prediction
model.transform(Study_data).show()
transformed = model.transform(Study_data)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+-----+
|               items| freq|
+--------------------+-----+
|                [t0]|79006|
|               [aim]|18029|
|           [aim, t0]|18029|
|          [patients]|13874|
|     [patients, aim]| 3678|
| [patients, aim, t0]| 3678|
|      [patients, t0]|13874|
|           [purpose]|10871|
| [purpose, patients]| 1850|
|[purpose, patient...| 1850|
|       [purpose, t0]|10871|
|          [evaluate]| 9971|
|[evaluate, patients]| 2131|
|[evaluate, patien...| 2131|
|     [evaluate, aim]| 3667|
| [evaluate, aim, t0]| 3667|
| [evaluate, purpose]| 2151|
|[evaluate, purpos...| 2151|
|      [evaluate, t0]| 9971|
|         [determine]| 7547|
+--------------------+-----+
only showing top 20 rows

+--------------------+----------+-------------------+------------------+--------------------+
|          antecedent|consequent|         confidence|              lift|             support|
+--------------------+----------+-------------------+------------------+------------------